#Inverse kinematics and resolved rate control

In this project, we will implement an inverse kinematics algorithm and controllers for the Kuka iiwa 14 robot using the results from Project 1.

The [Kuka iiwa 14 robot](https://www.kuka.com/en-us/products/robotics-systems/industrial-robots/lbr-iiwa) has 7 revolute joints and its kinematics is described in the picture below:

![](./kuka_kinematics.png "Kuka iiwa 14 Kinematic Model")

# Starting the visualization environment

The following code will start a visualization environment (click on the printed address to see the robot)

You need to run this only ONCE. Each time you run this cell you will get a new display environment (so you need to close the previous one!)

This should work out of the box on Google Colab and you local Jupyter Notebook (make sure you have installed the right libraries in your local computer if you do not use Colab).

In [ ]:
import numpy as np
import robot_visualizer
import time
from Kinematic_Information_for_KUKA_IIWA_14 import *

import matplotlib.pyplot as plt

robot_visualizer.start_robot_visualizer()

# Displaying an arbitrary configuration

As in the previous project, you can use the following function to display arbitrary configurations of the robot

In [ ]:
# here we display an arbitrary configuration of the robot
q = np.random.sample([7])
print(f'we show the configuration for the angles {q}')
robot_visualizer.display_robot(q)

In [ ]:
#### Functions

def J_B_in_parallel_frame_O_at_EE(theta):
    J_S = get_space_jacobian(theta)
    transform = forward_kinematics(theta)
    transform[:3,:3] = np.eye(3)
    inv_t = inverseT(transform)
    adj_T = getAdjoint(inv_t)
    J_B = np.matmul(adj_T, J_S)

    return(J_B)


## Inverse kinematics
* Function ``compute_IK_position`` that gets a desired end-effector 3D position (in spatial frame) and returns a vector of joint angles that solves the inverse kinematics problem is defined
* The file ``desired_end_effector_positions.npy`` contains a sequence of 10 desired end-effector positions. For all the positions attainable by the robot, compute an inverse kinematics solution. For the positions for which an inverse kinematics solution does not exist, what is the issue and how close can you get the end-effector to the desired position?
* A function ``compute_IK_position_nullspace`` that solves the inverse kinematics problem and additionally uses joint redundancy (i.e. the nullspace) to try and keep the joints close to the following configuration $[1,1,-1,-1,1,1,1]$ is defined. 
* This new function is used to reach the positions set in the file ``desired_end_effector_positions.npy`` and the outputs are compared.

In [ ]:
## a script to load the desired end effector positions and display each of them every second
## you maybe modify this script to test your code

# load the file
with open('desired_end_effector_positions.npy', 'rb') as f:
    desired_endeff = np.load(f)
    
# first we display the robot in 0 position
robot_visualizer.display_robot(np.zeros([7,1]))
    
# for each end-eff position
for i in range(desired_endeff.shape[1]):
    # displays the desired endeff position
    robot_visualizer.display_ball(desired_endeff[:,i])
    time.sleep(1.)

In [ ]:
# A Write a function ``compute_IK_position`` that gets a desired end-effector 
# 3D position (in spatial frame) and returns a vector of joint angles that solves the inverse kinematics problem
import numpy as np
from scipy.linalg import inv
# M_0 is the end-effector pose when all the joint angles are zero
theta = np.array([[0],[0],[0],[0],[0],[0],[0]])
epsilon = 10e-8
alpha = 0.1

def compute_IK_position(ee_pose_in_s):
    global theta, epsilon, alpha
    f_k = forward_kinematics(theta)
    error = ee_pose_in_s - f_k[0:3,3]
    
    for e in range(100):
        if np.linalg.norm(error) > epsilon: 
            J_b = J_B_in_parallel_frame_O_at_EE(theta)  
            J_b = J_b[3:7,:]
            fk = forward_kinematics(theta)
            error = ee_pose_in_s - fk[0:3,3]
            d_theta =  np.matmul(np.linalg.pinv(J_b), ((ee_pose_in_s).reshape(-1,1) - (fk[0:3,3]).reshape(-1,1)))
            theta = theta + alpha*d_theta
        else: 
            return(theta)
            
    return(theta)

for i in range(len(desired_endeff[0,:])):
    x_d = desired_endeff[:,i]
    I_K =  compute_IK_position(x_d)
    print(I_K) 
    print(forward_kinematics(I_K))
    print(x_d)


In [ ]:
#  Write a function ``compute_IK_position_nullspace`` that solves the inverse kinematics problem and additionally uses joint redundancy 
# (i.e. the nullspace) to try and keep the joints close to the following configuration $[1,1,-1,-1,1,1,1]$. Explain how you used the nullspace 
# to implement this function.
theta_d = np.array([[1],[1],[-1],[1],[1],[1],[1]])
theta = np.array([[0],[0],[0],[0],[0],[0],[0]])
def compute_IK_position_nullspace(ee_pose_in_s):
    global theta, epsilon
    f_k = forward_kinematics(theta)
    error = ee_pose_in_s - f_k[0:3,3]
    
    for e in range(100):
        if np.linalg.norm(error) > epsilon: 
            J_b = J_B_in_parallel_frame_O_at_EE(theta)
            J_b = J_b[3:7,:]
            fk = forward_kinematics(theta)
            error = ee_pose_in_s - fk[0:3,3]
            d_theta =  np.matmul(np.linalg.pinv(J_b), ((ee_pose_in_s).reshape(-1,1) - (fk[0:3,3]).reshape(-1,1))) + np.matmul((np.eye(7) - np.matmul(np.linalg.pinv(J_b), J_b)), (theta_d - theta))
            theta = theta + d_theta
        else: 
            return(theta)
    return(theta)

for i in range(len(desired_endeff[0,:])):
    x_d = desired_endeff[:,i]
    I_K =  compute_IK_position_nullspace(x_d)
    print(I_K)
    print(forward_kinematics(I_K))
    print(x_d)
  

## Joint control and joint trajectories generation
We would like the robot to go from its initial configuration to the desired end-effector positions (in spatial coordinates) $[0.7, 0.2,0.7]$ in 5 seconds and then to the configuration $[0.3, 0.5,0.9]$ during the following 5 seconds.

* Inverse kinematics solutions to reach both goals is computed
* A function ``get_point_to_point_motion`` that returns a desired position and velocity and takes as input the total motion duration T, the desired initial position and the desired final position is defined. The generated trajectory needs to ensure that at t=0 and t=T both the velocity and acceleration are 0. You can use this function to interpolate between desired positions in both joint and end-effector space.
* A function ``robot_controller`` is defined to move the robot from its initial configuration to reach the first goal (displayed in pink) at t=5 and the second goal ((in yellow) at t=10 by interpolating joint positions using the function ``get_point_to_point_motion`` we wrote above.
* The resulting joint simulated and desired positions and velocities are plotted
* The resulting end-effector positions and velocities are plotted

In [ ]:
T = 10.
end_effector_goal1 = np.array([[0.7], [0.2],[0.7]])
end_effector_goal2 = np.array([[0.3], [0.5],[0.9]])
theta_init = np.array([[0],[0],[0],[0],[0],[0],[0]])

## Compute inverse kinematics solutions to reach both goals

joint_angles_goal1 = compute_IK_position_nullspace(end_effector_goal1)
joint_angles_goal2 = compute_IK_position_nullspace(end_effector_goal2)


## this code is to save what the controller is doing for plotting and analysis after the simulation
global save_joint_positions, save_joint_velocities, save_t, ind
global save_des_joint_positions, save_des_joint_velocities
save_joint_positions = np.zeros([7,int(np.ceil(T / 0.001))+1])
save_joint_velocities = np.zeros_like(save_joint_positions)
save_des_joint_positions = np.zeros_like(save_joint_positions)
save_des_joint_velocities = np.zeros_like(save_joint_positions)
save_t = np.zeros([int(np.ceil(T / 0.001))+1])
ind=0
# end of saving code

##Write a function ``get_point_to_point_motion`` that returns a desired position and velocity and takes as input the total motion duration T, the desired initial position and the 
#desired final position. The generated trajectory needs to ensure that at t=0 and t=T both the velocity and acceleration are 0. You can use this function to interpolate between desired 
#positions in both joint and end-effector space.

t=0

def get_point_to_point_motion(T, theta_init, theta_goal):

    global t

    theta_des = theta_init + ((10/T**3)*t**3 - (15/T**4)*t**4 + (6/T**5)*t**5)*(theta_goal - theta_init)
        
    dtheta_des = ((30/T**3)*t**2 - (60/T**4)*t**3 + (30/T**5)*t**4)*(theta_goal - theta_init)
    
    return(theta_des, dtheta_des)



## Modify the ``robot_controller`` function below to move the robot from its initial configuration to reach the first goal (displayed in pink) at t=5 and the second goal ((in yellow) 
# at t=10 by interpolating joint positions using the function ``get_point_to_point_motion`` you wrote above.
# Plot the resulting joint simulated and desired positions and velocities
# Plot the resulting end-effector positions and velocities
    
def robot_controller(a, joint_positions, joint_velocities):
    global t
    t = a
    """A typical robot controller
        at every time t, this controller is called by the simulator. It receives as input
        the current joint positions and velocities and needs to return a [7,1] vector
        of desired torque commands
        
        As an example, the current controller implements a PD controller and at time = 5s
        it makes joint 2 and 3 follow sine curves
    """
    desired_joint_positions, desired_joint_velocities = get_point_to_point_motion(T/2, theta_init, joint_angles_goal1)

    # desired_joint_positions = np.zeros([7,1])
    # desired_joint_velocities = np.zeros([7,1])
    
    #When t>5. we generate sines for joint 2 and 3 as an example
    if a > 5:
        t = a - 5
        # desired_joint_positions[2] = 1. - np.cos(2*np.pi/5.*t)
        # desired_joint_velocities[2] = 2*np.pi/5. * np.sin(2*np.pi/5.*t)
        
        # desired_joint_positions[3] = .5 - 0.5*np.cos(2*np.pi/5.*t)
        # desired_joint_velocities[3] = np.p    i/5. * np.sin(2*np.pi/5.*t)


        desired_joint_positions, desired_joint_velocities = get_point_to_point_motion(T/2, joint_angles_goal1, joint_angles_goal2)

   
    # we compute the desired control commands using a PD controller
    P = np.array([100., 100., 100., 100., 100., 100., 100.])
    D = np.array([2.,2,2,2,2,2,2.])
    


    desired_joint_torques = np.diag(P) @ (desired_joint_positions - joint_positions)
    desired_joint_torques += np.diag(D) @ (desired_joint_velocities - joint_velocities)
    
    
    ## this code is to save what the controller is doing for plotting and analysis after the simulation
    global save_joint_positions, save_joint_velocities, save_t, ind
    global save_des_joint_positions, save_des_joint_velocities
    save_joint_positions[:,ind] = joint_positions[:,0]
    save_joint_velocities[:,ind] = joint_velocities[:,0]
    save_des_joint_positions[:,ind] = desired_joint_positions[:,0]
    save_des_joint_velocities[:,ind] = desired_joint_velocities[:,0]
    save_t[ind] = t
    ind += 1
    ## end of saving code
    
    return desired_joint_torques
        
robot_visualizer.display_ball(end_effector_goal1[:,0])
robot_visualizer.display_ball2(end_effector_goal2[:,0])
robot_visualizer.simulate_robot(robot_controller, T=T)

In [ ]:
# we plot the simulated vs. actual position of the robot
plt.figure(figsize=[9,12])
for i in range(7):
    plt.subplot(7,1,i+1)
    plt.plot(save_t, save_joint_positions[i,:])
    plt.plot(save_t, save_des_joint_positions[i,:])
    plt.ylim([-np.pi,np.pi])
    plt.ylabel(f'q {i}')
plt.xlabel('Desired vs. actual joint positions - Time [s]')

In [ ]:
# we plot the simulated vs. actual position of the robot
plt.figure(figsize=[9,12])
for i in range(7):
    plt.subplot(7,1,i+1)
    plt.plot(save_t, save_joint_velocities[i,:])
    plt.plot(save_t, save_des_joint_velocities[i,:])
    plt.ylim([-3,3])
    plt.ylabel(f'dq {i}')
plt.xlabel('Desired vs. actual joint velocities - Time [s]')

## End-effector control
As in previous part, we would like the robot to go from its initial configuration to the desired end-effector positions (in spatial coordinates) $[0.7, 0.2,0.7]$ in 5 seconds and then to the configuration $[0.3, 0.5,0.9]$ during the following 5 seconds.

* Defined a function ``robot_controller2`` function below to move the robot from its initial configuration to the first goal (reaching at t=5) and the second goal (t=10) by interpolating the desired end effector positions and directly mapping end-effector error to desired joint velocities (i.e. use P gains equal to 0 in joint space and do resolved-rate control).
* Plotted the resulting joint simulated and desired positions and velocities
* Plotted the resulting end-effector positions and velocities
* Compared results with previous part
* Added a nullspace term to optimize a desired configuration of your choice and discuss the results

In [ ]:
def p_inv(J):
    alpha = 10e-4
    inverse = np.matmul(np.transpose(J), np.linalg.inv(np.matmul(J, np.transpose(J)) + (alpha * np.eye(3))))

    return(inverse)
T = 10.

## this code is to save what the controller is doing for plotting and analysis after the simulation
global save_joint_positions, save_joint_velocities, save_t, ind
global save_des_joint_positions, save_des_joint_velocities
save_joint_positions = np.zeros([7,int(np.ceil(T / 0.001))+1])
save_joint_velocities = np.zeros_like(save_joint_positions)
save_des_joint_positions = np.zeros_like(save_joint_positions)
save_des_joint_velocities = np.zeros_like(save_joint_positions)
save_t = np.zeros([int(np.ceil(T / 0.001))+1])
ind=0
# end of saving code


def robot_controller2(a, joint_positions, joint_velocities):
    global t
    t = a
    """A typical robot controller
        at every time t, this controller is called by the simulator. It receives as input
        the current joint positions and velocities and needs to return a [7,1] vector
        of desired torque commands
        
        As an example, the current controller implements a PD controller and at time = 5s
        it makes joint 2 and 3 follow sine curves
    """
    

    # desired_joint_positions = np.zeros([7,1])
    # desired_joint_velocities = np.zeros([7,1])
    
    theta_init = np.array([[0],[0],[0],[0],[0],[0],[0]])
    init_ee_pose = forward_kinematics(theta_init)
    
    init_ee_pose = init_ee_pose[:3, -1].reshape(-1, 1)
    


    desired_end_effector_positions, desired_end_effector_velocities = get_point_to_point_motion(T/2, init_ee_pose, end_effector_goal1) 


    desired_joint_positions = compute_IK_position_nullspace(desired_end_effector_positions) 

    J_b = J_B_in_parallel_frame_O_at_EE(joint_positions)
    # here we will only use a D controller (i.e. on the desired joint velocities)
    # we increased the D gain for that purpose compared to the previous controller
    
    ##TODO - find the desired joint velocities
    P = np.eye(3) * 100
    D = np.array([4.,4,4,4,4,4,4.])
    desired_joint_velocities = p_inv(J_b[3:7,:]) @ ((P @ (desired_end_effector_positions) - forward_kinematics(joint_positions)[:3, -1].reshape(-1, 1) + desired_end_effector_velocities))
    
    if a > 5:
        t = a - 5

        desired_end_effector_positions, desired_end_effector_velocities = get_point_to_point_motion(T/2, end_effector_goal1, end_effector_goal2)

        desired_joint_positions = compute_IK_position_nullspace(desired_end_effector_positions)    

        P = np.eye(3) * 100
        D = np.array([4.,4,4,4,4,4,4.])
    
    ##TODO - find the desired joint velocities
        
     
        J_b = J_B_in_parallel_frame_O_at_EE(joint_positions)


        desired_joint_velocities = p_inv(J_b[3:7,:]) @ ((P @ (desired_end_effector_positions) - forward_kinematics(joint_positions)[:3, -1].reshape(-1, 1) + desired_end_effector_velocities))
       


    desired_joint_torques = np.diag(D) @ (desired_joint_velocities - joint_velocities)
    
    
    ## this code is to save what the controller is doing for plotting and analysis after the simulation
    global save_joint_positions, save_joint_velocities, save_t, ind
    global save_des_joint_positions, save_des_joint_velocities
    save_joint_positions[:,ind] = joint_positions[:,0]
    save_joint_velocities[:,ind] = joint_velocities[:,0]
    save_des_joint_positions[:,ind] = desired_joint_positions[:,0]
    save_des_joint_velocities[:,ind] = desired_joint_velocities[:,0]
    save_t[ind] = t
    ind += 1
    ## end of saving code
    
    return desired_joint_torques
        
robot_visualizer.display_ball(end_effector_goal1[:,0])
robot_visualizer.display_ball2(end_effector_goal2[:,0])
robot_visualizer.simulate_robot(robot_controller2, T=T)

In [ ]:
# we plot the simulated vs. actual position of the robot
plt.figure(figsize=[9,12])
for i in range(7):
    plt.subplot(7,1,i+1)
    plt.plot(save_t, save_joint_positions[i,:])
    plt.plot(save_t, save_des_joint_positions[i,:])
    plt.ylim([-np.pi,np.pi])
    plt.ylabel(f'q {i}')
plt.xlabel('Desired vs. actual joint positions - Time [s]')

## Impedance control and gravity compensation
As in parts 2 and 3, we would like the robot to go from its initial configuration to the desired end-effector positions (in spatial coordinates) $[0.7, 0.2,0.7]$ in 5 seconds and then to the configuration $[0.3, 0.5,0.9]$ during the following 5 seconds.

In the previous parts, a gravity compensation controller was running "in the background" in addition to the control law you were computing. In this question, we remove this and implement a complete impedance controller with gravity compensation.

YThe function ``robot_visualizer.rnea(q,dq,ddq)`` implements the Recursive Newton Euler Algorithm (RNEA). It takes as arguments a vector of positions, velocities and accelerations, and computes (and returns) the following $M(q) \cdot \ddot{q} + C(q,\dot{q}) + G(q)$

* Defined a function ``robot_controller3`` below to implement an impedance controller with gravity compensation (add a small amount of joint damping, using a joint-space D gain of 0.1). Use dthis controller to move the robot from its initial configuration to the first goal (reaching at t=5) and the second goal (t=10) by interpolating the desired end effector positions as in the previous questions.
* Plotted the resulting joint simulated and desired positions and velocities
* Plotted the resulting end-effector positions and velocities
* Compared the controller when the small joint samping is on or off.
* Compared results with parts 2 and 3.

In [ ]:
T = 10.

## this code is to save what the controller is doing for plotting and analysis after the simulation
global save_joint_positions, save_joint_velocities, save_t, ind
global save_des_joint_positions, save_des_joint_velocities
save_joint_positions = np.zeros([7,int(np.ceil(T / 0.001))+1])
save_joint_velocities = np.zeros_like(save_joint_positions)
save_des_joint_positions = np.zeros_like(save_joint_positions)
save_des_joint_velocities = np.zeros_like(save_joint_positions)
save_t = np.zeros([int(np.ceil(T / 0.001))+1])
ind=0
# end of saving code


def robot_controller3(a, joint_positions, joint_velocities):
    """A typical robot controller
        at every time t, this controller is called by the simulator. It receives as input
        the current joint positions and velocities and needs to return a [7,1] vector
        of desired torque commands
        
        As an example, the current controller implements a PD controller and at time = 5s
        it makes joint 2 and 3 follow sine curves
    """
    global t
    t = a

    theta_init = np.array([[0],[0],[0],[0],[0],[0],[0]])
    init_ee_pose = forward_kinematics(theta_init)
    
    init_ee_pose = init_ee_pose[:3, -1].reshape(-1, 1)
    


    desired_end_effector_positions, desired_end_effector_velocities = get_point_to_point_motion(T/2, init_ee_pose, end_effector_goal1)
    
    desired_joint_positions = compute_IK_position_nullspace(desired_end_effector_positions) 

    measured_positions = forward_kinematics(joint_positions)

    J_b = J_B_in_parallel_frame_O_at_EE(joint_positions)
    
    measured_velocities = J_b @ joint_velocities
    
    desired_joint_velocities = p_inv(J_b[3:7,:]) @ ((P @ (desired_end_effector_positions) - forward_kinematics(joint_positions)[:3, -1].reshape(-1, 1) + desired_end_effector_velocities))
    


    # desired_joint_positions = np.zeros([7,1])
    # desired_joint_velocities = np.zeros([7,1])
    
    # here we will only use the D controller to inject small joint damping
    P = np.eye(3) * 100
    D = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])


    if a > 5:
        t = a - 5

        desired_end_effector_positions, desired_end_effector_velocities = get_point_to_point_motion(T/2, end_effector_goal1, end_effector_goal2)
    
        desired_joint_positions = compute_IK_position_nullspace(desired_end_effector_positions) 

        measured_positions = forward_kinematics(joint_positions)

        J_b = J_B_in_parallel_frame_O_at_EE(joint_positions)
    
        measured_velocities = J_b @ joint_velocities
    
        desired_joint_velocities = p_inv(J_b[3:7,:]) @ ((P @ (desired_end_effector_positions) - forward_kinematics(joint_positions)[:3, -1].reshape(-1, 1) + desired_end_effector_velocities))
    


    # desired_joint_positions = np.zeros([7,1])
    # desired_joint_velocities = np.zeros([7,1])
    
    # here we will only use the D controller to inject small joint damping
    P = np.eye(3) * 100
    D = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

    
    ##TODO - implement gravity compensation and impedance control

    gravity = robot_visualizer.rnea(q,np.zeros(7,1),np.zeros(7,1))
    desired_joint_torques = J.T @ (P @ (desired_end_effector_positions - measured_positions) + D @ (desired_joint_velocities - measured_velocities)) + gravity
    
    
    ## this code is to save what the controller is doing for plotting and analysis after the simulation
    global save_joint_positions, save_joint_velocities, save_t, ind
    global save_des_joint_positions, save_des_joint_velocities
    save_joint_positions[:,ind] = joint_positions[:,0]
    save_joint_velocities[:,ind] = joint_velocities[:,0]
    save_des_joint_positions[:,ind] = desired_joint_positions[:,0]
    save_des_joint_velocities[:,ind] = desired_joint_velocities[:,0]
    save_t[ind] = t
    ind += 1
    ## end of saving code
    
    return desired_joint_torques
        
robot_visualizer.display_ball([0.7, 0.2,0.7])
robot_visualizer.display_ball2([0.3, 0.5,0.9])
robot_visualizer.simulate_robot(robot_controller3, T=T, gravity_comp = False)